## Are first babies more likely to be late?

Adapted from "Teaching statistical inference with resampling," Copyright 2018 Allen Downey
License: http://creativecommons.org/licenses/by/4.0/

In [ ]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(4)

Some people say that first babies are more likely to be late. And some people swear they're early! What's true?

The CDC runs the National Survey of Family Growth (NSFG), which "gathers information on family life, marriage and divorce, pregnancy, infertility, use of contraception, and men’s and women’s health." https://www.cdc.gov/nchs/nsfg/index.htm

In [ ]:
# Code to get data and into a Pandas Dataframe:
import nsfg

df = nsfg.ReadFemPreg()
df.shape

The file contains 13,593 rows, one for each pregnancy reported by a one of the survey respondents, and 244, one for each variable.

Here are the first few lines.

In [ ]:
df.head()

The variables we need are `outcome`, which indicates whether the pregnancy ended in a live birth, `birthord`, which indicates birth order, and `prglength`, which is pregnancy length in weeks.

From all live births, we can select first babies (`birthord==1`) and others:

In [ ]:
live = ...
firsts = ...
others = ...

In [ ]:
len(firsts), len(others)

Then we can get the list of pregnancy lengths for the two groups and compute their means:

In [ ]:
group1 = firsts.prglngth
group2 = others.prglngth

np.mean(group1), np.mean(group2)

What's the difference in means (in weeks) ?

In [ ]:
diff = ...

Convert the difference in means from weeks to hours

In [ ]:
diff * 7 * 24

So ...

## Hypothesis testing

The size of this "apparent effect" is small, and we can't tell whether it is real or the result of random sampling.  After all, we did not survey the entire population; we only surveyed a random sample.

There are two ways the sample might deviate from the population:

*  Systematic errors: The pregnancies included in the survey might be different from other pregnancies in a way that biases the results.

*  Sampling errors: The pregnancies lengths in one groups might be a little higher, or lower, than in the other group because of random variability.

We can never rule out the possibility of systematic errors, but usually we can test whether an apparent effect could be explained by random sampling.

Here's how:

1.  Choose a "test statistics" that measures the size of the effect; in this case, the test statistic we started with is the difference in mean pregnancy length.

2.  Use the data to make a model of the population under the assumption that there is actually no difference between the groups.  This assumption is called the "null hypothesis".

3.  Use the model to simulate the data collection process.

4.  Use the simulated data to compute the test statistic.

4.  Repeat steps 2-4 and collect the results.

5.  See how often the simulated test statistic exceeds the observed difference.

The following function computes the test statistic:

In [ ]:
def test_stat(data):
    group1, group2 = data
    return ...

Here's how we use it.

In [ ]:
data = group1, group2
actual = test_stat(data)

Now we need a model of the population under the assumption that these is actually no difference between the groups.

Well, if there's no difference, we can put the two groups together and shuffle them, then divide them at random into two groups with the same sizes.

That's what this function does:

In [ ]:
def run_model(data):
    group1, group2 = data
    pool = np.hstack((group1, group2))
    np.random.shuffle(pool)
    n = len(group1)
    return np.split(pool, [n])

Here's how we run it:

In [ ]:
run_model(data)

The result is a list of two arrays, which we can pass to `test_stat`:

In [ ]:
test_stat(run_model(data))

That's the result of one simulated experiment.

We can run the experiment 1000 times and collect the results.

In [ ]:
test_stat_dist = np.array([test_stat(run_model(data)) 
                           for i in range(1000)])
np.mean(test_stat_dist)

The result is the "sampling distribution of the test statistic under the null hypothesis".

Here's a function to plot the distribution of test stats:

In [ ]:
def plot_test_stats(test_stats):
    plt.xlabel('Difference in mean (weeks)')
    plt.title('Distribution of test stat under null hypothesis')
    return plot_hist(test_stats)

In [ ]:
def plot_hist(values, low=None, high=None):
    options = dict(alpha=0.5, color='C0')
    xs, ys, patches = plt.hist(values,
                               normed=True,
                               histtype='step', 
                               linewidth=3,
                               **options)
    
    
    plt.ylabel('Density')
    plt.tight_layout()
    return patches[0]

def fill_hist(low, high, patch):
    options = dict(alpha=0.5, color='C0')
    fill = plt.axvspan(low, high, 
                       clip_path=patch,
                       **options)

And here's what it looks like:

In [ ]:
plot_test_stats(test_stat_dist);

Now we can compute the probability that the test statistic, under the null hypothesis, exceeds the observed differences in the means.

This probability is called a "p-value".

In [ ]:
p_value = np.mean(test_stat_dist >= actual)

The following figure shows the p-value as the shaded area of the distribution above the observed value.

In [ ]:
def annotate(text, x, y, length):
    arrowprops = dict(width=1, headwidth=6, facecolor='black')
    plt.annotate(text,
                 xy=(x, y),
                 xytext=(x, y+length),
                 ha='center',
                 arrowprops=arrowprops)

In [ ]:
patch = plot_test_stats(test_stat_dist)
low = actual
high = np.max(test_stat_dist)
fill_hist(low, high, patch)
annotate('p-value', 1.2*actual, 1.0, 2)

## Different test statistics

What we computed in the previous section is the probability that first babies would be *later*, on average, under the null hypothesis. Depending on the context, we might also want to know the probability that first babies would be *earlier* on average.

We can include both possibilities by defining a new test statistic, the *absolute* difference in means.

In [ ]:
def test_stat(data):
    group1, group2 = data
    return abs(...)

Here's the observed difference with this test stat.

In [ ]:
actual = test_stat(data)

Since the actual difference is positive, its absolute value is the same.

We can run the simulated experiments with this test statistic, and print the p-value.

In [ ]:
test_stat_dist = np.array([test_stat(run_model(data)) 
                       for i in range(1000)])

p_value = np.mean(test_stat_dist >= actual)

Here's what the distribution looks like for this test statistic.

In [ ]:
patch = plot_test_stats(test_stat_dist)
low = actual
high = np.max(test_stat_dist)
fill_hist(low, high, patch)
annotate('p-value', 1.2*actual, 2, 3.5)